In [ ]:
%load_ext autoreload
%autoreload 2
# !pip install -e ../  # If not done yet...

In [ ]:
import os
import matplotlib.pyplot as plt
from pubrecon.data import DataFrame, ImagesData
from pubrecon.model import RCNN

In [ ]:
# General
data_path = "../data/in/"  # Where is your input data
work_path = "../data/out/"  # Where everything will be saved
seed = 1337  # Random seed
verbose = 1  # 0: no output; 1: normal informations; 2: e v e r y th i n g

# DataFrame
dataframe_pickle_path = os.path.join(work_path, "dataframe.pickle")  # Where will the DataFrame be saved
force_preparation = True  # Do you want to bypassed the saved DataFrame
subsamples = 512  # Number of samples to use for the DataFrame; -1: Use all of them

# ImagesData
imagesdata_pickle_path = os.path.join(work_path, 'imagesdata.pickle')  # Where will the ImagesData be saved
number_of_results = 2500  # How many samples will selective search use
iou_threshold = 0.85  # What is the percent of precision required
max_samples = 30  # How many class samples do you want
show_infos = False  # Show information for images output
show_labels = False  # Show labels for images output

# RCNN
model_and_weights_path = "../data/out/"  # Where will the model and weights be saved/loaded
loss = None  # Loss function; None: Use crossentropy
opt = None  # Optimization function; None: Use Adame
lr = 0.001  # Learning rate
epochs = 10  # Number of epochs
batch_size = 16
split_size = 0.10  # Test/Train proportion
checkpoint_path = os.path.join(work_path, 'checkpoint.h5')  # Where will the checkpoints be saved; None: No checkpoint (don't.)
early_stopping = False  # Should the learning stop if no more improvment is done
threshold = 0.80  # Threshold used for the recognition

#### If you did not generate data yet:

In [ ]:
# dataframe = DataFrame(data_path, pickle_path=dataframe_pickle_path)
# dataframe.prepare_data(force_preparation=force_preparation, subsamples=subsamples, verbose=verbose)
# dataframe.summary()

In [ ]:
# imagesdata = ImagesData(dataframe, pickle_path=imagesdata_pickle_path)
# # That part is quite long, beware!
# imagesdata.prepare_images_and_labels(number_of_results=number_of_results, iou_threshold=iou_threshold,
#                                      max_samples=max_samples, verbose=verbose)

In [ ]:
# import pickle
# # Save it for later.
# with open(imagesdata_pickle_path, 'wb') as fi:
#     pickle.dump(imagesdata, fi)

#### Else:

In [ ]:
import pickle
with open(imagesdata_pickle_path, 'rb') as fi:
    imagesdata = pickle.load(fi)

#### End if

In [ ]:
imagesdata.summary()

In [ ]:
from keras.optimizers import Adam, Nadam

losses = ['categorical_crossentropy']
optimizers = [Adam, Nadam]
lrs = [0.001 * x for x in range(1, 11)]
batch_sizes = [16, 32, 64, 128]

num_tests = len(losses) * len(optimizers) * len(lrs) * len(batch_sizes)
print("Total tests: {}.".format(num_tests))

In [ ]:
from tqdm import tqdm

tests = []
test = 0
for o in tqdm(optimizers):
    for l in tqdm(lrs, leave=False):
        for b in tqdm(batch_sizes, leave=False):
            test += 1
            print("test {}".format(test))
            print(o, l, b)
            model = RCNN(imagesdata, loss=None, opt=o, lr=l, verbose=0)
            model.train(epochs=5, batch_size=b, split_size=split_size, checkpoint_path=None,
                        early_stopping=None, verbose=1)
            tests.append([o, l, b, model.history()])
            
            with open(os.path.join(work_path, "tests.pickle"), 'wb') as fi:
                pickle.dump(tests, fi)